In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Make dataset

In [2]:
ohe = OneHotEncoder(sparse_output=False)

with open("constitution.txt", "r") as file:
    txt = file.read()

txt_arr = np.array(list(txt)).reshape(-1, 1)

chars = np.array(['\n', ' ', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3',
        '4', '5', '6', '7', '8', '9', ':', ';', 'A', 'B', 'C', 'D', 'E',
        'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
        'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
        'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
        'u', 'v', 'w', 'x', 'y', 'z']).reshape(-1, 1)

ohe.fit(chars)

sentences = txt.split("\n")[:-1]

batch_size = 5
batches = []

for sentence in sentences:
    encoded_sentence = ohe.transform(np.array(list(sentence)).reshape(-1, 1))
    encoded_sentence = np.concatenate([encoded_sentence, np.zeros((1, len(ohe.categories_[0])))])

    batches.append(encoded_sentence)

# batches.sort(key=len) # Make batches
len(ohe.categories_[0])

72

# RNN Implementation

In [13]:
class VanillaRNN:
    def __init__(
            self, 
            input_size, 
            hidden_size,
            output_size,
        ):
        # self.train_mode = True 
        self.loss = "cross_entropy"
        
        self.input_weights = np.random.normal(loc = 0, scale = 2 / (input_size + output_size), size=(input_size, hidden_size))
        self.hidden_weights = np.random.normal(loc = 0, scale = 1 / hidden_size, size=(hidden_size, hidden_size))
        self.bias = np.zeros((1, hidden_size))

        self.output_weights = np.random.normal(loc = 0, scale = 2 / (hidden_size + output_size), size=(hidden_size, output_size))
        self.output_bias = np.zeros((1, output_size))
        
        self.hidden_state = np.zeros((1, hidden_size))
        self.output = None

        self._input_weights_gradient = np.zeros_like(self.input_weights) # Or init by 0
        self._hidden_weights_gradient = np.zeros_like(self.hidden_weights)
        self._output_weights_gradient = np.zeros_like(self.output_weights)

        self._bias_gradient = 0
        self._output_bias_gradient = 0

        self._hidden_weights_gradient_recurrent = np.ones_like(self.hidden_weights)
        self._input_weights_gradient_recurrent = np.ones_like(self.input_weights)
        self._bias_gradient_recurrent = np.ones_like(self.bias)
        
    def forward(self, X):
        hidden_state = np.tanh(X @ self.input_weights + self.hidden_state @ self.hidden_weights + self.bias)
        output = hidden_state @ self.output_weights + self.output_bias

        return output, hidden_state

    def loss_function(self, y_pred, y_true):

        if self.loss == "cross_entropy":
            # + softmax
            self.probabilities_ = np.exp(y_pred) # - y_pred.max(axis=1, keepdims=True)) # avoid overflow
            self.probabilities_ = self.probabilities_ / self.probabilities_.sum(axis=1, keepdims=True)

            return -(np.log(self.probabilities_[np.arange(y_true.shape[0]), np.argmax(y_true, axis=1)])).mean()

    def __loss_function_gradient(self, y_pred, y_true):

        if self.loss == "cross_entropy":
            return 1 / len(y_true) * (self.probabilities_ - y_true)



    def fit(self, X, y):
        lr = 0.001

        global_loss = 0

        for i in range(len(X)):
            input_ = X[[i], :]
            label_ = y[[i], :]

            output_, hidden_state_ = self.forward(input_)

            loss = self.loss_function(output_, label_) # May be wanna print
            global_loss += loss

            gradient = self.__loss_function_gradient(output_, label_)

            self._output_weights_gradient += hidden_state_.T @ gradient
            self._output_bias_gradient += gradient # (1, output)

            recurrent_gradient = 1 - np.square(hidden_state_) # (1, hidden)

            self._input_weights_gradient += (input_.T @ (1 - np.square(hidden_state_)) +  ((1 - np.square(hidden_state_) @ self.hidden_weights.T) * self._input_weights_gradient_recurrent)) * (gradient @ self.output_weights.T)

            self._hidden_weights_gradient += (gradient @ self.output_weights.T) * (1 - np.square(hidden_state_)) * (self._hidden_weights_gradient_recurrent @ self.hidden_weights.T + self.hidden_state)

            self._bias_gradient += (gradient @ self.output_weights.T) * (1 - np.square(hidden_state_)) @ (self._bias_gradient_recurrent + self.hidden_weights.T)



            self._input_weights_gradient_recurrent *= ((1 - np.square(hidden_state_)) @ self.hidden_weights.T) * self._input_weights_gradient_recurrent # input, hidden
            self._hidden_weights_gradient_recurrent *= (1 - np.square(hidden_state_)) * (self._hidden_weights_gradient_recurrent @ self.hidden_weights.T + self.hidden_state)
            self._bias_gradient_recurrent *= (1 - np.square(hidden_state_)) @ (self._bias_gradient_recurrent + self.hidden_weights.T)
                
            self.hidden_state = hidden_state_
        
        self.input_weights -= lr * self._input_weights_gradient
        self.output_weights -= lr * self._output_weights_gradient
        self.output_bias -= lr * self._output_bias_gradient
        self.hidden_weights -= lr * self._hidden_weights_gradient
        self.bias -= lr * self._bias_gradient

        print(global_loss)

In [50]:
rnn = VanillaRNN(72, 142, 72)
X = batches[0]
rnn.fit(X, X)

C:\Users\Samvel\AppData\Local\Temp\ipykernel_12232\2200655831.py:85: RuntimeWarning: overflow encountered in multiply
  self._bias_gradient_recurrent *= (1 - np.square(hidden_state_)) @ (self._bias_gradient_recurrent + self.hidden_weights.T)
C:\Users\Samvel\AppData\Local\Temp\ipykernel_12232\2200655831.py:79: RuntimeWarning: invalid value encountered in matmul
  self._bias_gradient += (gradient @ self.output_weights.T) * (1 - np.square(hidden_state_)) @ (self._bias_gradient_recurrent + self.hidden_weights.T)


1402.5503151511314
